In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from utils import load_data, load_test_data, load_one_test_data
from utils import num_classes, epochs, batch_size

Using TensorFlow backend.


In [2]:
ensembling_models = {
    'Xception-Fine-Tune': {'img_size': 299},
    
}
#'DenseNet201-Fine-Tune': {'img_size': 224},
#    'InceptionResNetV2': {'img_size': 299}

In [3]:
#讀取訓練的一張圖片
for key in ensembling_models.keys():
    
    img_size = ensembling_models[key]['img_size']
    model = load_model('./{}.h5'.format(key))
    X_test, X_id = load_one_test_data("./data/examples/flower_classification/train/daisy/2077865117_9ed85191ae_n.jpg",Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)
    print(key)

Xception-Fine-Tune


In [6]:
for key in ensembling_models.keys():
    
    img_size = ensembling_models[key]['img_size']
    model = load_model('./{}.h5'.format(key))
    X_test, X_id = load_test_data(Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)
    print(key)

Xception-Fine-Tune


In [4]:
num_models = float(len(ensembling_models.keys()))
y_test_pred_prob = 0
for key in ensembling_models.keys():
    y_test_pred_prob += ensembling_models[key]['y_test_pred_prob']

y_test_pred_prob /= num_models

In [5]:
y_test_pred = y_test_pred_prob.argmax(axis=-1)

In [7]:
y_test_pred_prob

array([[9.9922085e-01, 4.8840098e-04, 7.2345043e-05, 1.1956050e-04,
        9.8877812e-05]], dtype=float32)

In [10]:
import numpy as np
import pandas as pd
y_test_pred_df = pd.DataFrame({'id': np.array(X_id), 'class':y_test_pred}).sort_values(by='id')

In [11]:
sub_data = pd.read_csv('submission.csv')
sub_data = sub_data.drop(["class"], axis=1)
y_test_pred_df_csv = pd.merge(sub_data, y_test_pred_df, how='left', on=['id'])

In [12]:
y_test_pred_df_csv.head()

,id,class
0,100080576_f52e8ee070_n,NaN
1,1008566138_6927679c8a,NaN
2,100930342_92e8746431_n,NaN
3,1022552002_2b93faf9e7_n,NaN
4,1022552036_67d33d5bd8_n,NaN


In [8]:
y_test_pred_df_csv.to_csv('./flowerTransfer.csv', index=False)